We are going to apply those classification algorithms that have given us a higher accuracy in the becars repository, this will allow us to increase the number of good purchases (MESES = 1). To do this, we focus on predicting new purchase opportunities that are not in our database, which means that we train on data that we have already trained and tested its high accuracy, but we will insert in an excel these new purchase opportunities

Lets first load required libraries:

In [214]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker

from pathlib import Path
data_path = Path('../data')

from sklearn import preprocessing
%matplotlib inline

Load Data From VENDIDOS1MESHISTORICOml2.xlsx File

In [215]:
df = pd.read_excel(data_path / 'VENDIDOS1MESHISTORICOml2.xlsx')

df.head()

,MARCA MODELO,MOTOR,CAMBIO,CV,COMBUSTIBLE,COLOR,PROVEEDOR,YEARS,PLAZAS,PCC,...,edad,mes,year,CTE.GARANTÍA,CTE..PREPARACIÓN,CTE.TOTAL,BF.NETO,GARANTIA,MESES,GARANTIA2
0,RENAULT MEGANE,1.6,MANUAL,75,GASOLINA,NaN,G.COSTA,10,NaN,956.6,...,NaN,1,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VOLVO S60,2.5,MANUAL,145,GASOLINA,NaN,PARTICULAR,7,NaN,7900.0,...,NaN,2,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VOLKSWAGEN PASSAT,1.9,MANUAL,130,DIESEL,NaN,LEASEPLAN,5,NaN,11200.0,...,NaN,2,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SEAT TOLEDO,1.9,MANUAL,110,DIESEL,NaN,LEASEPLAN,5,NaN,7300.0,...,NaN,3,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SKODA OCTAVIA,1.9,MANUAL,105,DIESEL,NaN,LEASEPLAN,3,NaN,10000.0,...,NaN,3,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Understanding the Data

MARCA MODELO== MAKE MODEL; MOTOR== cubic capacity; CAMBIO== type of vehicle speed change; CV== horsepower vehicle; COMBUSTIBLE== FUEL; COLOR== COLOUR; PROVEEDOR== SUPPLIER; YEARS== car age; PLAZAS== car seats; PCC== purchase price with commissions; PCOMPRA== purchase price without commissions; PVENTA== sale price; BRUTO== gross margin; NETO== net margin; KM== KM; C AUTO1== price commission; tipo== vehicle type; llantas== Tires; C.P Zip== code; localidad== locality; provincia== province; pais== country; nacionalidad== nationality; sexo== sex; edad== age; mes== sale month; year== sale year; CTE.GARANTÍA== cost in guarantee; CTE..PREPARACIÓN== preparation cost; CTE.TOTAL== total cost; BF.NETO== net profit; GARANTIA== Garantia=1 guaranteed car , Grantía=2 without guarantee; MESES== MESES=1 sold in 2 or less months, MESES=2 sold in more than 2 months; GARANTIA2== Garantia=si guaranteed car , Grantía=no without guarantee

what is the size of the dataset?

In [216]:
df.shape

(393, 34)

We remove those columns from the database that are not relevant in our study

In [217]:
df.drop(['GARANTIA','MOTOR','COLOR','PCOMPRA','BF.NETO','CTE.TOTAL','CTE.GARANTÍA','edad','sexo','nacionalidad','pais','provincia','localidad','C.P','C AUTO1','NETO','BRUTO','mes'], axis = 'columns', inplace=True)


In [218]:
df.head()

,MARCA MODELO,CAMBIO,CV,COMBUSTIBLE,PROVEEDOR,YEARS,PLAZAS,PCC,PVENTA,KM,tipo,llantas,year,CTE..PREPARACIÓN,MESES,GARANTIA2
0,RENAULT MEGANE,MANUAL,75,GASOLINA,G.COSTA,10,NaN,956.6,4500,109988,med,si,2008,NaN,NaN,NaN
1,VOLVO S60,MANUAL,145,GASOLINA,PARTICULAR,7,NaN,7900.0,10500,185390,gra,si,2008,NaN,NaN,NaN
2,VOLKSWAGEN PASSAT,MANUAL,130,DIESEL,LEASEPLAN,5,NaN,11200.0,15750,73790,gra,si,2008,NaN,NaN,NaN
3,SEAT TOLEDO,MANUAL,110,DIESEL,LEASEPLAN,5,NaN,7300.0,9900,97257,gra,si,2008,NaN,NaN,NaN
4,SKODA OCTAVIA,MANUAL,105,DIESEL,LEASEPLAN,3,NaN,10000.0,12600,92481,gra,si,2008,NaN,NaN,NaN


#5 tops rows after removing nan values

In [219]:
df = df.dropna()
df.head()

,MARCA MODELO,CAMBIO,CV,COMBUSTIBLE,PROVEEDOR,YEARS,PLAZAS,PCC,PVENTA,KM,tipo,llantas,year,CTE..PREPARACIÓN,MESES,GARANTIA2
99,VOLKSWAGEN POLO,MANUAL,75,GASOLINA,MAVISA,12,5.0,1805.0,4500,91431,peq,si,2016,2089.46,2.0,si
100,OPEL INSIGNIA,MANUAL,130,DIESEL,ARVAL,4,5.0,10821.0,13300,74210,gra,si,2016,307.60,1.0,si
101,CITROEN C4,MANUAL,110,GASOLINA,AUTO1,10,5.0,2250.0,5300,82294,med,no,2016,986.07,1.0,si
102,AUDI A4,MANUAL,130,DIESEL,MUNDOMOVIL,12,5.0,4000.0,7500,184488,gra,si,2016,967.89,1.0,si
103,RENAULT TRAFIC,MANUAL,115,DIESEL,ARVAL,6,6.0,10900.0,13700,84172,fur,no,2016,821.60,1.0,si


return the number of missing values in the data set

In [220]:
df.isnull().sum()

MARCA MODELO        0
CAMBIO              0
CV                  0
COMBUSTIBLE         0
PROVEEDOR           0
YEARS               0
PLAZAS              0
PCC                 0
PVENTA              0
KM                  0
tipo                0
llantas             0
year                0
CTE..PREPARACIÓN    0
MESES               0
GARANTIA2           0
dtype: int64

In [221]:
df = pd.DataFrame(df)

df_columns = df.columns

In [222]:
predictors = df[df_columns[df_columns != 'MESES']] # all columns except MESES 
target = df['MESES'] # MESES  column

In [223]:
predictors.head()

,MARCA MODELO,CAMBIO,CV,COMBUSTIBLE,PROVEEDOR,YEARS,PLAZAS,PCC,PVENTA,KM,tipo,llantas,year,CTE..PREPARACIÓN,GARANTIA2
99,VOLKSWAGEN POLO,MANUAL,75,GASOLINA,MAVISA,12,5.0,1805.0,4500,91431,peq,si,2016,2089.46,si
100,OPEL INSIGNIA,MANUAL,130,DIESEL,ARVAL,4,5.0,10821.0,13300,74210,gra,si,2016,307.60,si
101,CITROEN C4,MANUAL,110,GASOLINA,AUTO1,10,5.0,2250.0,5300,82294,med,no,2016,986.07,si
102,AUDI A4,MANUAL,130,DIESEL,MUNDOMOVIL,12,5.0,4000.0,7500,184488,gra,si,2016,967.89,si
103,RENAULT TRAFIC,MANUAL,115,DIESEL,ARVAL,6,6.0,10900.0,13700,84172,fur,no,2016,821.60,si


In [224]:
target.head()

99     2.0
100    1.0
101    1.0
102    1.0
103    1.0
Name: MESES, dtype: float64

Use one hot encoding technique to conver categorical varables to binary variables and append them to the predictors.

In [225]:
predictors = pd.get_dummies(predictors, columns = ["MARCA MODELO"])
predictors = pd.get_dummies(predictors, columns = ["PROVEEDOR"])
predictors = pd.get_dummies(predictors, columns = ["CAMBIO"])
predictors = pd.get_dummies(predictors, columns = ["COMBUSTIBLE"])
predictors = pd.get_dummies(predictors, columns = ["tipo"])
predictors = pd.get_dummies(predictors, columns = ["llantas"])
predictors = pd.get_dummies(predictors, columns = ["GARANTIA2"])


Feature selection.
Lets defind feature sets, X:
Step1 save x to later insert that modified excel with these new purchase opportunities, for predicting them!!!

In [227]:
x = predictors
# Step1 x.to_excel(data_path /'x.xlsx')
x[0:5]

,CV,YEARS,PLAZAS,PCC,PVENTA,KM,year,CTE..PREPARACIÓN,MARCA MODELO_AUDI A4,MARCA MODELO_AUDI A6,...,tipo_mon,tipo_monp,tipo_peq,tipo_pickup,tipo_sub,tipo_tt,llantas_no,llantas_si,GARANTIA2_no,GARANTIA2_si
99,75,12,5.0,1805.0,4500,91431,2016,2089.46,0,0,...,0,0,1,0,0,0,0,1,0,1
100,130,4,5.0,10821.0,13300,74210,2016,307.60,0,0,...,0,0,0,0,0,0,0,1,0,1
101,110,10,5.0,2250.0,5300,82294,2016,986.07,0,0,...,0,0,0,0,0,0,1,0,0,1
102,130,12,5.0,4000.0,7500,184488,2016,967.89,1,0,...,0,0,0,0,0,0,0,1,0,1
103,115,6,6.0,10900.0,13700,84172,2016,821.60,0,0,...,0,0,0,0,0,0,1,0,0,1


What are our lables?

In [228]:
y = target
y[0:5]

99     2.0
100    1.0
101    1.0
102    1.0
103    1.0
Name: MESES, dtype: float64

## K-NEAREST METHOD with k=14

The first time we use the notebook don't run data normalization. The important thing is to save x to insert there the x_test  purchase opportunities to predict, always maintaining at all times the train set that has provided us our highest accuracy as we see at becars repository.

In [229]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (235, 128) (235,)
Test set: (59, 128) (59,)


Classification
K nearest neighbor (KNN)
Import library
Classifier implementing the k-nearest neighbors vote.

In [230]:
from sklearn.neighbors import KNeighborsClassifier

Step2 we save X_test in excel to later modify in x excel those values to introduce the new purchase opportunities to predict.

In [231]:
X_test[0:5]
#Step2 X_test.to_excel(data_path / 'X_test.xlsx')

,CV,YEARS,PLAZAS,PCC,PVENTA,KM,year,CTE..PREPARACIÓN,MARCA MODELO_AUDI A4,MARCA MODELO_AUDI A6,...,tipo_mon,tipo_monp,tipo_peq,tipo_pickup,tipo_sub,tipo_tt,llantas_no,llantas_si,GARANTIA2_no,GARANTIA2_si
251,140,11,5.0,4300.0,7990,155272,2017,1448.08,0,0,...,1,0,0,0,0,0,0,1,0,1
314,75,9,5.0,3700.0,5500,44098,2015,114.41,0,0,...,0,0,1,0,0,0,0,1,0,1
325,200,4,5.0,12900.0,16500,119225,2015,871.77,0,0,...,0,0,0,0,0,1,1,0,0,1
276,100,15,5.0,2477.0,4790,60849,2020,341.02,0,0,...,0,0,1,0,0,0,0,1,0,1
389,125,11,5.0,4500.0,7800,103186,2022,0.00,0,0,...,0,0,0,0,0,0,0,1,0,1


We insert our excel x with the modifications in the positions that the X_test has given us to predict those new purchase opportunities.

In [ ]:
#Step3 x = pd.read_excel(data_path / 'x.xlsx')

In [232]:
x = preprocessing.StandardScaler().fit(x).transform(x.astype(float))

In [233]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (235, 128) (235,)
Test set: (59, 128) (59,)


Training
Lets start the algorithm with k=14 for now

In [234]:
k = 14
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh


KNeighborsClassifier(n_neighbors=14)

In [235]:
yhat = neigh.predict(X_test)
yhat[0:5]

array([1., 1., 1., 1., 1.])

We see that the first 5 are good purchases. The best accuracy in our becars repository was with classifier implementing the k-nearest neighbors 0.7288135593220338 with k= 14

## Modeling (SVM with Scikit-learn)

Although it should also be noted that for f1 score modeling with SVM in becars repository for class 1 (good sales) is a high 0.78. Let's see how it predicts new buying opportunities.

In [239]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 

SVC()

In [240]:
yhat = clf.predict(X_test)
yhat [0:5]

array([1., 1., 2., 1., 1.])

Taking into account only the first 5 predictions:predictions in X_test differ from those of K-NEAREST METHOD with k=14 in the first five vehicles only in the third, we recommend the purchase of the other four vehicles.